# Two federal programs at odds
## School lunch debt in Durham Public Schools and competition between CEP and Title I

In [71]:
# Load packages
require(ggplot2)
require(dplyr)
require(tidyr)
require(knitr)
require(stringr)

# Don't load strings as factors
options('stringsAsFactors'=FALSE)

# Load lunch debt and 2018-19 banding data
debt = read.csv('./data/meal_debt.csv')
bands = read.csv('./data/banding_1718.csv')

# Load CEP eligibility data
# File names as list
cep.list = list.files('./data') %>%
    grep('cep', ., value=TRUE) %>%
    paste('./data/', ., sep='')

# Read files
cep.dat = lapply(cep.list, function(x) read.csv(x))

# Add year
year = c('2014-15', '2015-16', '2016-17', '2017-18', '2018-19')
for(i in 1:length(cep.dat)) {
  cep.dat[[i]]$year = year[i]
}
                 
# Merge to one dataset
cep = do.call(plyr::rbind.fill, cep.dat)
                 
# Add missing school numbers for 2014-15 CEP eligibility data
school_no_key = cep %>%
  filter(!is.na(school_no)) %>%
  select(school_name, school_no) %>%
  unique()
cep$school_no[cep$year=="2014-15"] = school_no_key$school_no[school_no_key$school_name %in% cep$school_name[cep$year=='2014-15']]

                 
# Filter to Durham
cep = cep %>%
    filter(lea_no == 320)
                 
# Control for data types
cep$school_no = cep$school_no %>% as.factor()
debt$school_no = debt$school_no %>% as.factor()
                 
# Merge lunch debt and CEP data
df = left_join(cep, debt, by = c('school_name', 'school_no', 'year'))

In [72]:
df

school_name,lea_name,lea_no,isp,eligible,year,school_no,participating,enrollment,unpaid
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
Bethesda Elementary,Durham Public Schools,320,0.519,X,2014-15,360,NA,NA,NA
Brogden Middle,Durham Public Schools,320,0.449,X,2014-15,364,NA,NA,NA
Burton Elementary,Durham Public Schools,320,0.628,X,2014-15,384,NA,NA,NA
C C Spaulding Elementary,Durham Public Schools,320,0.750,X,2014-15,388,NA,NA,NA
Club Boulevard Elementary,Durham Public Schools,320,0.313,,2014-15,392,NA,NA,NA
E K Powe Elementary,Durham Public Schools,320,0.425,X,2014-15,396,NA,NA,NA
Eastway Elementary,Durham Public Schools,320,0.715,X,2014-15,304,NA,NA,NA
Eno Valley Elementary,Durham Public Schools,320,0.546,X,2014-15,306,NA,NA,NA
Fayetteville Street Elementary,Durham Public Schools,320,0.674,X,2014-15,308,NA,NA,NA
